In [9]:
#──────────────────────────────────────────────────────────────
# 1. 라이브러리
import pandas as pd, numpy as np, xgboost as xgb
from tqdm import tqdm

#──────────────────────────────────────────────────────────────
# 2. 데이터 로드 및 전처리 (관측 + 예측)
obs  = pd.read_csv('데이터_분석과제_7_기상관측데이터_2401_2503.csv')
pred = pd.read_csv('데이터_분석과제_7_기상예측데이터_2401_2503.csv')
obs.columns  = ['datetime', 'humidity_obs', 'temp_obs', 'pressure_obs']
pred.columns = ['datetime', 'solar_rad_pred', 'humidity_pred', 'absolute_humidity_pred', 'temp_pred', 'pressure_pred']

# 단위 통일 (pressure: mmHg → hPa)
obs['pressure_obs'] = obs['pressure_obs'] * 1.33322
obs['datetime']     = pd.to_datetime(obs['datetime'])
pred['datetime']    = pd.to_datetime(pred['datetime'])

df = pd.merge(obs, pred, on='datetime')

# 오차 변수 생성
df['temp_error']     = df['temp_pred']     - df['temp_obs']
df['humidity_error'] = df['humidity_pred'] - df['humidity_obs']
df['pressure_error'] = df['pressure_pred'] - df['pressure_obs']

# 시간 파생변수
df['hour']  = df['datetime'].dt.hour
df['month'] = df['datetime'].dt.month
df['day']   = df['datetime'].dt.day
df['year']  = df['datetime'].dt.year

df['hour_sin']  = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos']  = np.cos(2 * np.pi * df['hour'] / 24)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

#──────────────────────────────────────────────────────────────
# 3. 이상치 제거
def get_outlier_indices(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return series[(series < lower) | (series > upper)].index

out_idx = set(get_outlier_indices(df['pressure_error']))
out_idx |= set(get_outlier_indices(df['temp_error']))
out_idx |= set(get_outlier_indices(df['humidity_error']))

drop_out = df.drop(index=out_idx).reset_index(drop=True)
ori_data = df.copy()

#──────────────────────────────────────────────────────────────
# 4. 데이터 분할 (train/val/test)
train = drop_out.query("'2024-01-01' <= datetime < '2025-01-01'")
val   = ori_data.query("'2025-01-01' <= datetime < '2025-03-01'")
test  = ori_data.query("'2025-03-01' <= datetime < '2025-04-01'")

#──────────────────────────────────────────────────────────────
# 5. 피처셋 (Residual: *_pred 포함, *_error 타깃)
targets = ["humidity_error", "pressure_error", "temp_error"]
drop_cols = ["datetime","humidity_obs","pressure_obs","temp_obs"] + targets
all_feats = [c for c in ori_data.columns if c not in drop_cols]

sincos = ["hour_sin","hour_cos","month_sin","month_cos"]
feature_sets = {
    "all_features": all_feats,
    "drop_sincos" : [c for c in all_feats if c not in sincos],
}
target_fs_map = {
    "humidity_error": "all_features",
    "temp_error"    : "all_features",
    "pressure_error": "drop_sincos",
}

#──────────────────────────────────────────────────────────────
# 6. 확정 파라미터 (Grid Search 결과)
best_params = {
    "humidity_error": dict(max_depth=2, learning_rate=0.05, subsample=0.4,
                            colsample_bytree=1.0, n_estimators=100,
                            reg_lambda=0, reg_alpha=0.0),
    "temp_error":     dict(max_depth=4, learning_rate=0.10, subsample=0.4,
                            colsample_bytree=0.8, n_estimators=100,
                            reg_lambda=10, reg_alpha=0.5),
    "pressure_error": dict(max_depth=3, learning_rate=0.05, subsample=0.4,
                            colsample_bytree=1.0, n_estimators=100,
                            reg_lambda=0,  reg_alpha=0.0),
}

tree_m = "gpu_hist"
pred_m = "gpu_predictor"

#──────────────────────────────────────────────────────────────
# 7. 모델 학습 (TRAIN만 사용)
final_models = {}
for tgt, fs_name in target_fs_map.items():
    feats = feature_sets[fs_name]

    dtr = xgb.DMatrix(train[feats], label=train[tgt])
    bst = xgb.train({**best_params[tgt], "tree_method":tree_m,
                     "predictor":pred_m, "verbosity":0},
                    dtr, num_boost_round=best_params[tgt]["n_estimators"])

    final_models[tgt] = bst
    print(f"✅ Finished {tgt}")

#──────────────────────────────────────────────────────────────
# 8. Test 예측 (오차→관측값 복원) & sAError 계산 (+mmHg 변환)
def single_metric(y_true, y_pred):
    rmse = np.sqrt(((y_true - y_pred) ** 2).mean())
    mae  = np.abs(y_true - y_pred).mean()
    return 0.5 * (rmse + mae)

weights = {"humidity_obs":0.3, "pressure_obs":0.2, "temp_obs":0.5}

pred_df = test[["datetime", "humidity_obs", "temp_obs", "pressure_obs",
                "humidity_pred", "temp_pred", "pressure_pred"]].copy()

# 1) *_error 예측
for tgt, fs_name in target_fs_map.items():
    feats   = feature_sets[fs_name]
    err_pred = final_models[tgt].predict(xgb.DMatrix(test[feats]))
    pred_df[f"{tgt}_pred"] = err_pred

# 2) 관측치 복원: *_obs_pred = *_pred - *_error_pred
pred_df["humidity_obs_pred"] = pred_df["humidity_pred"] - pred_df["humidity_error_pred"]
pred_df["temp_obs_pred"]     = pred_df["temp_pred"]     - pred_df["temp_error_pred"]
pred_df["pressure_obs_pred"] = pred_df["pressure_pred"] - pred_df["pressure_error_pred"]

# 3) pressure 단위 변환 (hPa→mmHg) 및 sAError 계산(옵셔널)
pred_df["pressure_obs_mmHg"]      = pred_df["pressure_obs"] / 1.33322
pred_df["pressure_obs_pred_mmHg"] = pred_df["pressure_obs_pred"] / 1.33322

err_score = {}
err_score["humidity_obs"] = single_metric(pred_df["humidity_obs"].values, pred_df["humidity_obs_pred"].values)
err_score["temp_obs"]     = single_metric(pred_df["temp_obs"].values, pred_df["temp_obs_pred"].values)
err_score["pressure_obs"] = single_metric(pred_df["pressure_obs_mmHg"].values, pred_df["pressure_obs_pred_mmHg"].values)

# 가중합 지표
sA = sum(weights[t]*err_score[t] for t in weights)

print("\n── Test sAError (½·RMSE+MAE, obs 기준, pressure mmHg 변환) ──")
for t in weights:
    if t == "pressure_obs":
        print(f"{t:<16s}: {err_score[t]:6.6f} (mmHg)")
    else:
        print(f"{t:<16s}: {err_score[t]:6.6f}")
print(f"\n✅ Weighted sAError = {sA:8.5f}")

# 4) 제출용 CSV 저장: datetime, temp, humidity, pressure만
submit_df = pred_df[["datetime", "temp_obs_pred", "humidity_obs_pred", "pressure_obs_pred_mmHg"]].copy()
submit_df.columns = ["datetime", "temp", "humidity", "pressure"]

out_path = f"./obs_preds_xgboost_test_march.csv"
submit_df.to_csv(out_path, index=False)
print(f"\n📄 Saved submission csv to {out_path} (datetime, temp, humidity, pressure)")

#──────────────────────────────────────────────────────────────
# 9. 하이퍼파라미터 요약
print("\n🛠️  Hyperparameters (fixed)")
for tgt, p in best_params.items():
    print(f"\n[{tgt}]")
    for k, v in p.items():
        print(f"  {k:<17s}: {v}")


✅ Finished humidity_error
✅ Finished temp_error
✅ Finished pressure_error

── Test sAError (½·RMSE+MAE, obs 기준, pressure mmHg 변환) ──
humidity_obs    : 9.275018
pressure_obs    : 0.542253 (mmHg)
temp_obs        : 1.407145

✅ Weighted sAError =  3.59453

📄 Saved submission csv to ./obs_preds_xgboost_test_march.csv (datetime, temp, humidity, pressure)

🛠️  Hyperparameters (fixed)

[humidity_error]
  max_depth        : 2
  learning_rate    : 0.05
  subsample        : 0.4
  colsample_bytree : 1.0
  n_estimators     : 100
  reg_lambda       : 0
  reg_alpha        : 0.0

[temp_error]
  max_depth        : 4
  learning_rate    : 0.1
  subsample        : 0.4
  colsample_bytree : 0.8
  n_estimators     : 100
  reg_lambda       : 10
  reg_alpha        : 0.5

[pressure_error]
  max_depth        : 3
  learning_rate    : 0.05
  subsample        : 0.4
  colsample_bytree : 1.0
  n_estimators     : 100
  reg_lambda       : 0
  reg_alpha        : 0.0
